# In this project i scrape data to complete my dataset and to create a recommendation system hybride

# script pour scrapper les donnees  

In [39]:
import pandas as pd
from bs4 import BeautifulSoup as bt
import time as t
import re
from lxml import html
import selenium
import time as t
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException
import pymongo

In [40]:
import mysql.connector as connection
import pandas as pd
try:
    mydb = connection.connect(host="localhost", database = 'book_system',user="root", passwd="root",use_pure=True)
    query = "Select isbn from Document where type='book';"
    result_dataFrame = pd.read_sql(query,mydb)
    mydb.close() #close the connection
except Exception as e:
    mydb.close()
    print(str(e))

In [41]:
result_dataFrame.isnull().sum()

isbn    0
dtype: int64

In [42]:
result_dataFrame.dropna(inplace=True)
result_dataFrame.isnull().sum()

isbn    0
dtype: int64

In [44]:
result_dataFrame

,isbn
0,9780446675963
1,9780446670647
2,9781576751602
3,9781576750742
4,9780684849133
...,...
6235,9781563273605
6236,9781119742715
6237,
6238,9781523085170


In [45]:
indexNames = result_dataFrame[ result_dataFrame['isbn'] == '' ].index

In [46]:
result_dataFrame.drop(indexNames , inplace=True)

In [47]:
result_dataFrame[ result_dataFrame['isbn'] != '' ]

,isbn
0,9780446675963
1,9780446670647
2,9781576751602
3,9781576750742
4,9780684849133
...,...
6234,9781138438071
6235,9781563273605
6236,9781119742715
6238,9781523085170


In [48]:

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
# driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/home/ay/.wdm/drivers/chromedriver/linux64/91.0.4472.101/chromedriver] found in cache
<ipython-input-48-542889ae2b36>:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)


In [49]:
def info_Book(ID, ISBN, title, author, pulisher, date_pub, count_rating, count_review, description, count_page, Genres, average_rating):
    
    dic_book = dict()
    dic_book['ID']          = ID
    dic_book['ISBN']        = ISBN
    dic_book['title']       = title
    dic_book['author']      = author
    dic_book['pulisher']    = pulisher
    dic_book['date_pub']    = date_pub
    dic_book['count_rating']= count_rating
    dic_book['count_review']= count_review
    dic_book['description'] = description
    dic_book['count_page']  = count_page
    dic_book['Genres']              = Genres
    dic_book['average_rating']      = average_rating
    
    return dic_book

In [51]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["SytemRecommandation"]
mycol = mydb["books_scrapper"]

In [50]:
mycol

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'SytemRecommandation'), 'books_scrapper')

In [52]:
def get_Data(ISBN):
    search_url="https://www.goodreads.com/search?utf8=%E2%9C%93&query={query}"
    driver.get(search_url.format(query=ISBN))
    t.sleep(1)
    try:
        button_close = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/button')
        button_close.click()
    except:
        pass
    
    try:
        read_more = driver.find_element_by_xpath('//*[@id="description"]/a')
        read_more.click()
    except Exception as e:
        print(str(e))
    
    
    try:
        get_details_book = driver.find_element_by_xpath('//*[@id="bookDataBoxShow"]')
        get_details_book.click()
    except:
        pass
    t.sleep(2)
    page = driver.page_source
    soup = bt(page)
    tree = html.fromstring(page)
    
    # first part all inormations about book
    try:
        ID = re.findall('[0-9]+', driver.current_url)[0]
        ISBN = ISBN
        title = re.sub('\s+', ' ',tree.xpath('//*[@id="bookTitle"]/text()')[0].replace("\n", ''))
        author  = tree.xpath('//*[@id="bookAuthors"]/span[2]/div/a/span/text()')[0]
        pulisher = re.sub('\s+', ' ', tree.xpath('//*[@id="details"]/div[2]/text()')[0].split('\n')[3])
        date_pub = re.sub('\s+', ' ', tree.xpath('//*[@id="details"]/div[2]/text()')[0].split('\n')[2])
        # ratingCount -------------------------------------------------------------------------
        count_rating = soup.find('meta', {'itemprop': "ratingCount"})
        if type(count_rating) != 'NoneType':
            count_rating = soup.find('meta', {'itemprop': "ratingCount"})['content']
        else:
            count_rating = 0
        # --------------------------------------------------------------------------------------
        # reviewCount -------------------------------------------------------------------------
        count_review = soup.find('meta', {'itemprop': "reviewCount"})
        if type(count_review) != 'NoneType':
            count_review = soup.find('meta', {'itemprop': "reviewCount"})['content']
        else:
            count_review = 0
        # --------------------------------------------------------------------------------------
        description = soup.find('div', {'class':'readable stacked'})
        if type(description) != 'NoneType':
            description = description.text.replace('\n', ' ')
        else:
            description = ""
        C = soup.find('span', {'itemprop':'numberOfPages'})
        count_page = 0
        if type(C) != 'NoneType':
            count_page = re.findall('[0-9]+',C.text)[0]  
        genres = []
        stacked = soup.findAll('div', {'class':'stacked'})
        lenght_ = 0
        for elem in stacked:
            elements = elem.find('div', {'class':'bigBoxContent containerWithHeaderContent'})
            if elements != [] and elements != None:
                lenght_ = len(elements.findAll('div', {'class':'elementList'}))
        for  i in range(lenght_):
            g = dict()
            g['gendre'] = tree.xpath('/html/body/div[2]/div[3]/div[1]/div[2]/div[5]/div[6]/div/div[2]/div')[0][i][0][0].text
            g['count_users'] = re.sub('\s+', " ",tree.xpath('/html/body/div[2]/div[3]/div[1]/div[2]/div[5]/div[6]/div/div[2]/div')[0][i][1][0].text)
            genres.append(g)
        # average_rating
        avgeratings = tree.xpath('/html/body/div[2]/div[3]/div[1]/div[2]/div[4]/div[1]/div[2]/div[2]/span[2]/text()')
        if len(avgeratings) == 1:
            print(avgeratings[0].replace("\n",''))
        else:
            avgeratings = -1
        book = info_Book(ID, ISBN, title, author, pulisher, date_pub, count_rating, count_review, description, count_page, genres, avgeratings)
        x = mycol.insert_one(book)
        print(x)  
    except:

        print('errors')

In [55]:
i = 0
for isbn in result_dataFrame['isbn'].values:
    if i>14:  
        get_Data(isbn)
        with open('cuurent_position_book_scrapper.txt', 'w') as f:
            f.write(str(i))
    i+=1

  3.72
  3.76
  3.64
  2.67
errors
errors
  3.00
  2.67
  3.95
  3.95
  3.86
errors
  2.00
  4.06
  4.03
  4.00
  2.33
errors
errors
  3.04
  4.18
  3.89
errors
  3.00
  3.89
  3.73
  3.22
errors
  3.08
errors
  3.50
  3.78
  3.76
errors
errors
  3.00
errors
  3.51
  3.64
  2.67
  3.72
  3.84
  3.64
  2.00
  3.00
errors
errors
  3.96
  3.69
  3.72
  3.33
  3.63
  3.75
  4.50
  3.00
  3.60
errors
errors
  4.15
errors
  4.67
  3.79
  3.85
  3.56
errors
errors
  3.42
  3.66
  3.68
  3.45
  3.82
  3.32
  3.80
  4.08
errors
  0.00
  4.00
  3.85
errors
errors
  3.96
  3.70
  4.10
  3.25
errors
errors
errors
  4.33
  2.38
errors
errors
  3.36
  3.18
errors
  4.18
errors
  3.73
  3.77
  3.00
  3.33
  3.44
  3.61
errors
errors
  3.74
  3.40
  2.50
  4.00
  4.00
  3.50
  3.33
errors
  3.88
  3.83
errors
errors
  4.11
  4.19
  3.71
  3.42
  3.50
  3.99
  3.58
  3.92
  4.01
  3.56
  4.03
errors
  3.92
errors
errors
  3.60
  3.63
  4.10
  4.20
  4.06
  3.33
  4.14
errors
  3.95
  4.20
  4.05
  3.69

TimeoutException: Message: timeout: Timed out receiving message from renderer: -0.001
  (Session info: headless chrome=91.0.4472.77)


In [53]:
pwd

'/home/ay/Stage/PFE__/Implementaion/pratique'

In [54]:
cat cuurent_position_book_scrapper.txt

14

In [ ]:
12588

In [28]:
len(result_dataFrame['isbn'].values)

22714

In [56]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["SytemRecommandation"]
mycol = mydb["books_iferu"]

In [9]:
isb = []
for i in mycol.find({'ISBN':"English"}, {"_id":0, "title":1}):
    isb.append(i['title'])

In [11]:
# isb

In [20]:
not_saved = []
k=0
def get_Data_(ISBN):
    search_url="https://www.goodreads.com/search?utf8=%E2%9C%93&query={query}"
    driver.get(search_url.format(query=ISBN))
    t.sleep(1)
    try:
        click_titre = driver.find_element_by_xpath('/html/body/div[2]/div[3]/div[1]/div[2]/div[2]/table/tbody/tr/td[2]/a')
        click_titre.click()
        t.sleep(1)
    except:
        pass
    try:
        button_close = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/button')
        button_close.click()
    except:
        pass
    try:
        read_more = driver.find_element_by_xpath('//*[@id="description"]/a')
        read_more.click()
    except:
        pass
    try:
        get_details_book = driver.find_element_by_xpath('//*[@id="bookDataBoxShow"]')
        get_details_book.click()
    except:
        pass
        # 
    t.sleep(1)
    page = driver.page_source
    soup = bt(page)
    tree = html.fromstring(page)
    
    # first part all inormations about book
    try:
        ID = re.findall('[0-9]+', driver.current_url)[0]

        ISBN =   re.sub("\s+", "", tree.xpath('/html/body/div[2]/div[3]/div[1]/div[2]/div[4]/div[1]/div[2]/div[5]/div[3]/div[1]/div[1]/div[2]/text()')[0].replace('\n', ""))
        if len(ISBN) >= 15:
            ISBN =    re.sub("\s+", "", tree.xpath('/html/body/div[2]/div[3]/div[1]/div[2]/div[4]/div[1]/div[2]/div[5]/div[3]/div[1]/div[2]/div[2]/text()')[0].replace('\n', ""))
        
        title = re.sub('\s+', ' ',tree.xpath('//*[@id="bookTitle"]/text()')[0].replace("\n", ''))
        author  = tree.xpath('//*[@id="bookAuthors"]/span[2]/div/a/span/text()')[0]
        pulisher = re.sub('\s+', ' ', tree.xpath('//*[@id="details"]/div[2]/text()')[0].split('\n')[3])
        date_pub = re.sub('\s+', ' ', tree.xpath('//*[@id="details"]/div[2]/text()')[0].split('\n')[2])

        # ratingCount -------------------------------------------------------------------------
        count_rating = soup.find('meta', {'itemprop': "ratingCount"})
        if type(count_rating) != 'NoneType':
            count_rating = soup.find('meta', {'itemprop': "ratingCount"})['content']
        else:
            count_rating = 0
        # --------------------------------------------------------------------------------------
        # reviewCount -------------------------------------------------------------------------
        count_review = soup.find('meta', {'itemprop': "reviewCount"})
        if type(count_review) != 'NoneType':
            count_review = soup.find('meta', {'itemprop': "reviewCount"})['content']
        else:
            count_review = 0
        # --------------------------------------------------------------------------------------
        description = soup.find('div', {'class':'readable stacked'})
        if type(description) != 'NoneType':
            description = description.text.replace('\n', ' ')
        else:
            description = ""

        C = soup.find('span', {'itemprop':'numberOfPages'})
        count_page = 0
        if type(C) != 'NoneType':
            count_page = re.findall('[0-9]+',C.text)[0]  

        genres = []
        stacked = soup.findAll('div', {'class':'stacked'})
        lenght_ = 0
        for elem in stacked:
            elements = elem.find('div', {'class':'bigBoxContent containerWithHeaderContent'})
            if elements != [] and elements != None:
                lenght_ = len(elements.findAll('div', {'class':'elementList'}))
        for  i in range(lenght_):
            g = dict()
            g['gendre'] = tree.xpath('/html/body/div[2]/div[3]/div[1]/div[2]/div[5]/div[6]/div/div[2]/div')[0][i][0][0].text
            g['count_users'] = re.sub('\s+', " ",tree.xpath('/html/body/div[2]/div[3]/div[1]/div[2]/div[5]/div[6]/div/div[2]/div')[0][i][1][0].text)
            genres.append(g)
        # average_rating
        avgeratings = tree.xpath('/html/body/div[2]/div[3]/div[1]/div[2]/div[4]/div[1]/div[2]/div[2]/span[2]/text()')
        if len(avgeratings) == 1:
            print(avgeratings[0].replace("\n",''))
        else:
            avgeratings = -1
        book = info_Book(ID, ISBN, title, author, pulisher, date_pub, count_rating, count_review, description, count_page, genres, avgeratings)
        x = mycol.insert_one(book)
        print(x, ISBN)  
    except:
        not_saved.append(ISBN)
        print('errors')

In [ ]:
i=0
for j in isb:
    if i> 919:
        get_Data_(j)
        with open('cu_english_book.txt', 'w') as f:
            f.write(str(i))
    i+=1

In [23]:
cat cu_english_book.txt

2120